# Sentiment Analysis

The following notebook is a guided exploration of the word2vec & lstm (version of `rnn`) neural network.

Credit to [sasakitetsuya](https://www.kaggle.com/sasakitetsuya) for the notebook.

Modifications to instructions and code are indicated via a note from me: (*Note*: ...)

In [1]:
!pip install gensim
!pip install ntlk
!pip install plotly

  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/d3/68/373da90a8b241e2603707c7aa4c8f47829a72729c6e9497f2bc604fa6a6a/gensim-4.3.2-cp39-cp39-win_amd64.whl.metadata
  Obtaining dependency information for smart-open>=1.8.1 from https://files.pythonhosted.org/packages/fc/d9/d97f1db64b09278aba64e8c81b5d322d436132df5741c518f3823824fae0/smart_open-6.4.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------

ERROR: Could not find a version that satisfies the requirement ntlk (from versions: none)
ERROR: No matching distribution found for ntlk


In [ ]:
import numpy as np 
import pandas as pd 

from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

from textblob import TextBlob

from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

import time

from keras.layers import Dense, Input, Flatten, LSTM, Bidirectional,Embedding, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential, load_model
from keras import losses
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.callbacks import EarlyStopping

According to the article "How to Get Your Customers to Recommend Your Brand to Others" (https://www.quicksprout.com/how-to-get-your-customers-to-recommend-your-brand-to-others/)

"Don’t underestimate the power of a recommendation. Research shows 83% of consumers say they trust recommendations from family and friends. When your brand gets recommended to someone, they are likely to give it a shot."

So in this notebook, I tried to find how we can predict that they will recommned or not by analyzing their review text by

NLTK - Tokenize and Valence Aware Dictionary and sEntiment Reasoner (VADER)
TF-IDF & Logistic Regression
Word2Vec & Logistic Regression
RNN and LSTM
I refered the notebook 'Twitter Sentiment Analysis - EDA and ML/DL' by Mr.Seungbum Lim.

( https://www.kaggle.com/code/seungbumlim/twitter-sentiment-analysis-eda-and-ml-dl )

In [ ]:
df=pd.read_csv('dresses.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df = df.rename(columns={'recommend_index ': 'recommend_index'})

In [ ]:
label=['1', '2','3', '4', '5']
plt.pie(df.groupby('rating')['rating'].count(),autopct="%.1f%%",labels=label)
plt.show()

In [ ]:
label2=['Non Recommend', 'Recommend']
plt.pie(df.groupby('recommend_index')['recommend_index'].count(),autopct="%.1f%%",labels=label2)
plt.show()

In [ ]:
pd.DataFrame(df.groupby('rating')['recommend_index'].value_counts()).unstack().plot.barh(figsize=(5,5),stacked=True)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=10,labels=label2)
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
sns.histplot(data=df, x="age", hue="rating")

In [ ]:
plt.figure(figsize=(15,8))
sns.histplot(data=df, x="age", hue="recommend_index")

In [ ]:
pd.DataFrame(df.groupby('division_name')['rating'].value_counts()).unstack().plot.barh(figsize=(5,5),stacked=True)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=10)
plt.show()

In [ ]:
pd.DataFrame(df.groupby('division_name')['recommend_index'].value_counts()).unstack().plot.barh(figsize=(5,5),stacked=True)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=10)
plt.show()

In [ ]:
pd.DataFrame(df.groupby('department_name')['rating'].value_counts()).unstack().plot.barh(figsize=(5,5),stacked=True)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=10)
plt.show()

In [ ]:
pd.DataFrame(df.groupby('department_name')['recommend_index'].value_counts()).unstack().plot.barh(figsize=(5,5),stacked=True)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=10)
plt.show()

In [ ]:
pd.DataFrame(df.groupby('class_name')['rating'].value_counts()).unstack().plot.barh(figsize=(5,5),stacked=True)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=10)
plt.show()

In [ ]:
pd.DataFrame(df.groupby('class_name')['recommend_index'].value_counts()).unstack().plot.barh(figsize=(5,5),stacked=True)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=10)
plt.show()

In [ ]:
df=df.dropna()
df = df.reset_index()

In [ ]:
num_1 = df[df['recommend_index']== 0]['review_text'].apply(lambda x: len(x.split()))
num_2 = df[df['recommend_index']== 1]['review_text'].apply(lambda x: len(x.split()))
plt.figure(figsize=(12,6))
sns.kdeplot(num_1, shade=True, color = 'red').set_title('Kernel distribution of number of words')
sns.kdeplot(num_2, shade=True, color = 'blue')

plt.legend(labels=[0,1])

In [ ]:
df['class_name'].unique()

In [ ]:
def clean_text(d):
    pattern = r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', d)
    return text

names = ['Jeans', 'Blouses', 'Skirts', 'Pants', 'Swim', 'Sweaters', 'Knits',
       'Fine gauge', 'Jackets', 'Dresses', 'Shorts', 'Lounge',
       'Outerwear', 'Intimates', 'Legwear', 'Sleep', 'Trend', 'Layering',
       'nan', 'Casual bottoms', 'Chemises']

# In 'Twitter Sentiment Analysis - EDA and ML/DL', the name of class are not relevant to the sentiment analysis and added to the list of stop words.
# So I followed it.
def clean_stopword(d):
    stop_words = stopwords.words('english')
    for name in names:
        stop_words.append(name)
    return " ".join([w.lower() for w in d.split() if w.lower() not in stop_words and len(w) > 1])

def tokenize(d):
    return word_tokenize(d)

In [ ]:
df['final_text']= df.review_text.apply(clean_text).apply(clean_stopword).apply(tokenize)
df.final_text.head()

In [ ]:
print(" ".join(df.final_text[0]),'\n')
print(" ".join(df.final_text[1]),'\n')
print(" ".join(df.final_text[100]),'\n')
print(" ".join(df.final_text[1000]),'\n')
print(" ".join(df.final_text[10000]))